<div align='center'>

# OVERVIEW
</div>

This notebook aims to clarify **whether or not any of the suggested Ledger & Accounting services are acceptable for use for our Customer Support Center system**, as defined in the [`customer_support_rep_persona_01.md` persona document](../../customer_rep_persona_store/customer_support_rep_persona_01.md).

The suggested services are listed below:

1. **PostgreSQL** (Relational Database for Internal Bookkeeping)
2. **Beancount** (Double-entry Open-source Ledger System)

The criteria for qualifying a suggestion as a tool is defined below:

1. Python-based.
2. Open-source (free, avoid freemium as much as possible) customer-facing **and** producer-facing access.
3. Rate limit restrictions allow for appropriate edge-case testing.
4. Supports core architectural choice for LangChain ecosystem (e.g. supports asynchronous backends, etc).
5. I can currently access the tool as a human (i.e. confirmation that it is operable as a human, and has not been deprecated/restricted discreetly).
6. I can perform its expected basic operations.

## Pre-Notebook Initialization

1. Install PostgreSQL locally or via Docker: `docker run --name postgres -e POSTGRES_PASSWORD=postgres -p 5432:5432 -d postgres`
2. Install Beancount: `pip install beancount`
3. Install Fava (web interface for Beancount): `pip install fava`
4. Create a test database in PostgreSQL for ledger operations.

<u>Notes</u>:
- PostgreSQL serves as the primary database for customer balances and transaction records.
- Beancount provides double-entry accounting capabilities with plain-text ledger files.

# Required Modules


In [ ]:
import os
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
load_dotenv()
from decimal import Decimal
from datetime import datetime


## PostgreSQL


<ol>
<li> Python-based: <b>Available</b> (via psycopg2, asyncpg, SQLAlchemy)
<li> Open-source: <b>Free</b> (PostgreSQL License - liberal open source)

<li> Maximum Rate Limit Restriction:
<ul>
<li> Self-hosted: <b>No rate limits</b> (depends on infrastructure and connection pooling).
<li> Default max connections: 100 (configurable).
</ul>

<li> <b>Supports Core Architectural Choice</b> (asyncpg for async operations, SQLAlchemy for ORM)
<li> Service is confirmed to <b>still be active & accessible</b>
</ol>

### Expected Basic Operations

1. [Connect to Database](https://www.psycopg.org/docs/usage.html#basic-module-usage).
2. [Create Ledger Table](https://www.postgresql.org/docs/current/sql-createtable.html).
3. [Insert Transaction Entry](https://www.postgresql.org/docs/current/sql-insert.html).
4. [Query Customer Balance](https://www.postgresql.org/docs/current/sql-select.html).
5. [Update Balance](https://www.postgresql.org/docs/current/sql-update.html).
6. [Flag Discrepancy](https://www.postgresql.org/docs/current/sql-update.html).

**<u>Notes:</u>**
1. PostgreSQL supports ACID transactions essential for financial data integrity.
2. The `psycopg2` library is the most popular PostgreSQL adapter for Python.
3. For async operations, `asyncpg` provides high-performance async PostgreSQL access.


### Setup

PostgreSQL provides robust relational database capabilities. We'll create a simple client to manage ledger operations.


In [ ]:
# Class Definition for PostgreSQL Ledger operations

class Simple_PostgreSQL_Ledger_Client():
    
    def __init__(
        self,
        host: str = None,
        port: int = 5432,
        database: str = None,
        user: str = None,
        password: str = None
    ):
        self.host = host or os.getenv("POSTGRES_HOST", "localhost")
        self.port = port
        self.database = database or os.getenv("POSTGRES_DB", "ledger_db")
        self.user = user or os.getenv("POSTGRES_USER", "postgres")
        self.password = password or os.getenv("POSTGRES_PASSWORD", "postgres")
        self.SUCCESS_MSG = "[INFO]: Operation performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        self.connection = None
        
    def connect(self):
        """Establish connection to PostgreSQL database"""
        try:
            self.connection = psycopg2.connect(
                host=self.host,
                port=self.port,
                database=self.database,
                user=self.user,
                password=self.password
            )
            print(self.SUCCESS_MSG, "Connected to database.")
            return True
        except Exception as e:
            print(self.ERROR_MSG, e)
            return False
    
    def disconnect(self):
        """Close database connection"""
        if self.connection:
            self.connection.close()
            print(self.SUCCESS_MSG, "Disconnected from database.")
    
    def create_ledger_tables(self):
        """Create tables for customer accounts and transactions"""
        try:
            cursor = self.connection.cursor()
            
            # Create accounts table
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS accounts (
                    account_id SERIAL PRIMARY KEY,
                    customer_id VARCHAR(50) UNIQUE NOT NULL,
                    balance DECIMAL(18, 8) DEFAULT 0.00,
                    currency VARCHAR(10) DEFAULT 'USD',
                    status VARCHAR(20) DEFAULT 'active',
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)
            
            # Create transactions table (double-entry style)
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS transactions (
                    transaction_id SERIAL PRIMARY KEY,
                    reference_id VARCHAR(100) UNIQUE NOT NULL,
                    debit_account VARCHAR(50) NOT NULL,
                    credit_account VARCHAR(50) NOT NULL,
                    amount DECIMAL(18, 8) NOT NULL,
                    currency VARCHAR(10) DEFAULT 'USD',
                    description TEXT,
                    status VARCHAR(20) DEFAULT 'completed',
                    has_discrepancy BOOLEAN DEFAULT FALSE,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)
            
            self.connection.commit()
            cursor.close()
            print(self.SUCCESS_MSG, "Ledger tables created.")
            return True
        except Exception as e:
            self.connection.rollback()
            print(self.ERROR_MSG, e)
            return False
    
    def create_account(self, customer_id: str, initial_balance: Decimal = Decimal("0.00")):
        """Create a new customer account"""
        try:
            cursor = self.connection.cursor()
            cursor.execute(
                """INSERT INTO accounts (customer_id, balance) VALUES (%s, %s)
                   ON CONFLICT (customer_id) DO NOTHING
                   RETURNING account_id""",
                (customer_id, initial_balance)
            )
            result = cursor.fetchone()
            self.connection.commit()
            cursor.close()
            print(self.SUCCESS_MSG, f"Account created for {customer_id}.")
            return result[0] if result else None
        except Exception as e:
            self.connection.rollback()
            print(self.ERROR_MSG, e)
            return None
    
    def get_balance(self, customer_id: str):
        """Query customer balance"""
        try:
            cursor = self.connection.cursor()
            cursor.execute(
                "SELECT balance, currency FROM accounts WHERE customer_id = %s",
                (customer_id,)
            )
            result = cursor.fetchone()
            cursor.close()
            if result:
                print(self.SUCCESS_MSG)
                return {"balance": float(result[0]), "currency": result[1]}
            return None
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def record_transaction(self, reference_id: str, debit_account: str, credit_account: str, 
                          amount: Decimal, description: str = ""):
        """Record a double-entry transaction"""
        try:
            cursor = self.connection.cursor()
            cursor.execute(
                """INSERT INTO transactions 
                   (reference_id, debit_account, credit_account, amount, description)
                   VALUES (%s, %s, %s, %s, %s)
                   RETURNING transaction_id""",
                (reference_id, debit_account, credit_account, amount, description)
            )
            result = cursor.fetchone()
            
            # Update balances
            cursor.execute(
                "UPDATE accounts SET balance = balance - %s, updated_at = CURRENT_TIMESTAMP WHERE customer_id = %s",
                (amount, debit_account)
            )
            cursor.execute(
                "UPDATE accounts SET balance = balance + %s, updated_at = CURRENT_TIMESTAMP WHERE customer_id = %s",
                (amount, credit_account)
            )
            
            self.connection.commit()
            cursor.close()
            print(self.SUCCESS_MSG, f"Transaction {reference_id} recorded.")
            return result[0] if result else None
        except Exception as e:
            self.connection.rollback()
            print(self.ERROR_MSG, e)
            return None
    
    def flag_discrepancy(self, transaction_id: int):
        """Flag a transaction as having a discrepancy"""
        try:
            cursor = self.connection.cursor()
            cursor.execute(
                "UPDATE transactions SET has_discrepancy = TRUE WHERE transaction_id = %s",
                (transaction_id,)
            )
            self.connection.commit()
            cursor.close()
            print(self.SUCCESS_MSG, f"Transaction {transaction_id} flagged for discrepancy.")
            return True
        except Exception as e:
            self.connection.rollback()
            print(self.ERROR_MSG, e)
            return False
    
    def reconcile_entries(self, customer_id: str):
        """Reconcile all entries for a customer account"""
        try:
            cursor = self.connection.cursor()
            
            # Get sum of credits and debits
            cursor.execute("""
                SELECT 
                    COALESCE(SUM(CASE WHEN credit_account = %s THEN amount ELSE 0 END), 0) as credits,
                    COALESCE(SUM(CASE WHEN debit_account = %s THEN amount ELSE 0 END), 0) as debits
                FROM transactions WHERE status = 'completed'
            """, (customer_id, customer_id))
            
            result = cursor.fetchone()
            cursor.close()
            
            if result:
                credits = float(result[0])
                debits = float(result[1])
                calculated_balance = credits - debits
                
                # Get stored balance
                stored = self.get_balance(customer_id)
                stored_balance = stored["balance"] if stored else 0
                
                print(self.SUCCESS_MSG)
                return {
                    "customer_id": customer_id,
                    "credits": credits,
                    "debits": debits,
                    "calculated_balance": calculated_balance,
                    "stored_balance": stored_balance,
                    "is_reconciled": abs(calculated_balance - stored_balance) < 0.01
                }
            return None
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None


### 1. Connect to Database

Establishes a connection to the PostgreSQL database.


In [ ]:
# PostgreSQL Client Init

pg_client = Simple_PostgreSQL_Ledger_Client()

# Connecting to database
pg_client.connect()


### 2. Create Ledger Tables

Creates the accounts and transactions tables for double-entry bookkeeping.


In [ ]:
# Creating ledger tables

pg_client.create_ledger_tables()


### 3. Create Customer Account

Creates a new customer account with an initial balance.


In [ ]:
# Creating customer accounts

pg_client.create_account("CUST001", Decimal("1000.00"))
pg_client.create_account("CUST002", Decimal("500.00"))


### 4. Query Customer Balance

Retrieves the current balance for a customer account.


In [ ]:
# Querying customer balance

balance_info = pg_client.get_balance("CUST001")
print(f"Customer CUST001 Balance: {balance_info}")


### 5. Record Transaction Entry

Records a double-entry transaction between two accounts.


In [ ]:
# Recording a transaction (transfer from CUST001 to CUST002)

import uuid
tx_ref = f"TXN-{uuid.uuid4().hex[:8].upper()}"

pg_client.record_transaction(
    reference_id=tx_ref,
    debit_account="CUST001",
    credit_account="CUST002",
    amount=Decimal("100.00"),
    description="Transfer payment"
)


### 6. Reconcile Entries

Reconciles all entries for a customer account to verify balance accuracy.


In [ ]:
# Reconciling entries for CUST001

reconciliation = pg_client.reconcile_entries("CUST001")
print(f"Reconciliation Result: {reconciliation}")


## Beancount


<ol>
<li> Python-based: <b>Available</b> (native Python implementation)
<li> Open-source: <b>Free</b> (GNU GPLv2 License)

<li> Maximum Rate Limit Restriction:
<ul>
<li> Local processing: <b>No rate limits</b> (depends on file size and system resources).
<li> Designed for personal/business accounting at scale.
</ul>

<li> <b>Supports Core Architectural Choice</b> (Python API for programmatic access)
<li> Service is confirmed to <b>still be active & accessible</b>
</ol>

### Expected Basic Operations

1. [Parse Beancount File](https://beancount.github.io/docs/beancount_scripting_plugins.html).
2. [Add Transaction Entry](https://beancount.github.io/docs/beancount_language_syntax.html#transactions).
3. [Query Account Balances](https://beancount.github.io/docs/beancount_query_language.html).
4. [Generate Reports](https://beancount.github.io/docs/beancount_query_language.html).
5. [Validate Ledger](https://beancount.github.io/docs/running_beancount_and_generating_reports.html).
6. [Export to Other Formats](https://beancount.github.io/docs/beancount_scripting_plugins.html).

**<u>Notes:</u>**
1. Beancount uses plain-text files for ledger storage, enabling version control.
2. Fava provides a modern web interface for viewing Beancount ledgers.
3. Strict double-entry accounting with automatic balance verification.


### Setup

Beancount provides a plain-text double-entry accounting system. We'll create a client to manage ledger operations programmatically.


In [ ]:
# Class Definition for Beancount Ledger operations

class Simple_Beancount_Client():
    
    def __init__(self, ledger_path: str = "ledger.beancount"):
        self.ledger_path = ledger_path
        self.SUCCESS_MSG = "[INFO]: Operation performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        
    def create_ledger_file(self, title: str = "Customer Support Center Ledger"):
        """Create a new Beancount ledger file with basic structure"""
        try:
            header = f'''; {title}
; Generated: {datetime.now().strftime("%Y-%m-%d")}

option "title" "{title}"
option "operating_currency" "USD"

; Account Definitions
2024-01-01 open Assets:Bank:Checking USD
2024-01-01 open Assets:Receivables USD
2024-01-01 open Liabilities:Payables USD
2024-01-01 open Equity:Opening-Balances USD
2024-01-01 open Income:Services USD
2024-01-01 open Expenses:Operations USD

; Customer Accounts
2024-01-01 open Assets:Customers:CUST001 USD
2024-01-01 open Assets:Customers:CUST002 USD

'''
            with open(self.ledger_path, 'w') as f:
                f.write(header)
            
            print(self.SUCCESS_MSG, f"Ledger file created: {self.ledger_path}")
            return True
        except Exception as e:
            print(self.ERROR_MSG, e)
            return False
    
    def add_transaction(self, date: str, payee: str, narration: str, 
                       postings: list):
        """Add a transaction to the ledger file
        
        Args:
            date: Transaction date (YYYY-MM-DD)
            payee: Payee name
            narration: Transaction description
            postings: List of (account, amount, currency) tuples
        """
        try:
            transaction = f'\n{date} * "{payee}" "{narration}"\n'
            for account, amount, currency in postings:
                if amount is not None:
                    transaction += f'  {account}  {amount:.2f} {currency}\n'
                else:
                    transaction += f'  {account}\n'
            
            with open(self.ledger_path, 'a') as f:
                f.write(transaction)
            
            print(self.SUCCESS_MSG, f"Transaction added: {narration}")
            return True
        except Exception as e:
            print(self.ERROR_MSG, e)
            return False
    
    def parse_ledger(self):
        """Parse the ledger file and return entries"""
        try:
            from beancount import loader
            from beancount.parser import parser
            
            entries, errors, options = loader.load_file(self.ledger_path)
            
            if errors:
                print(f"[WARNING]: Found {len(errors)} errors in ledger")
                for error in errors[:5]:
                    print(f"  - {error}")
            
            print(self.SUCCESS_MSG, f"Parsed {len(entries)} entries")
            return entries, errors, options
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None, None, None
    
    def get_account_balance(self, account_pattern: str):
        """Get balance for accounts matching a pattern"""
        try:
            from beancount import loader
            from beancount.core import realization
            from beancount.core import getters
            
            entries, errors, options = loader.load_file(self.ledger_path)
            real_root = realization.realize(entries)
            
            # Find matching accounts
            balances = {}
            accounts = getters.get_accounts(entries)
            
            for account in accounts:
                if account_pattern in account:
                    real_account = realization.get(real_root, account)
                    if real_account:
                        balance = real_account.balance
                        balances[account] = str(balance)
            
            print(self.SUCCESS_MSG)
            return balances
        except Exception as e:
            print(self.ERROR_MSG, e)
            return {}
    
    def validate_ledger(self):
        """Validate the ledger for errors"""
        try:
            from beancount import loader
            
            entries, errors, options = loader.load_file(self.ledger_path)
            
            if not errors:
                print(self.SUCCESS_MSG, "Ledger is valid - no errors found")
                return True, []
            else:
                print(f"[WARNING]: Ledger has {len(errors)} errors")
                return False, errors
        except Exception as e:
            print(self.ERROR_MSG, e)
            return False, [str(e)]
    
    def read_ledger_content(self):
        """Read and return the raw ledger content"""
        try:
            with open(self.ledger_path, 'r') as f:
                content = f.read()
            print(self.SUCCESS_MSG)
            return content
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None


### 1. Create Ledger File

Creates a new Beancount ledger file with account definitions.


In [ ]:
# Beancount Client Init

bc_client = Simple_Beancount_Client("sample_ledger.beancount")

# Creating ledger file
bc_client.create_ledger_file("Customer Support Center Ledger")


### 2. Add Transaction Entry

Adds a double-entry transaction to the ledger.


In [ ]:
# Adding transactions

# Initial deposit to CUST001
bc_client.add_transaction(
    date="2024-01-15",
    payee="Initial Deposit",
    narration="Opening balance for CUST001",
    postings=[
        ("Assets:Customers:CUST001", 1000.00, "USD"),
        ("Equity:Opening-Balances", -1000.00, "USD")
    ]
)

# Transfer from CUST001 to CUST002
bc_client.add_transaction(
    date="2024-01-20",
    payee="Transfer",
    narration="Payment from CUST001 to CUST002",
    postings=[
        ("Assets:Customers:CUST002", 100.00, "USD"),
        ("Assets:Customers:CUST001", -100.00, "USD")
    ]
)


### 3. Parse Ledger File

Parses the ledger file and returns all entries.


In [ ]:
# Parsing the ledger file

entries, errors, options = bc_client.parse_ledger()
if entries:
    print(f"Total entries: {len(entries)}")


### 4. Query Account Balances

Retrieves balances for customer accounts.


In [ ]:
# Querying customer account balances

customer_balances = bc_client.get_account_balance("Customers")
print("Customer Account Balances:")
for account, balance in customer_balances.items():
    print(f"  {account}: {balance}")


### 5. Validate Ledger

Validates the ledger file for errors.


In [ ]:
# Validating the ledger

is_valid, validation_errors = bc_client.validate_ledger()
print(f"Ledger valid: {is_valid}")


### 6. View Ledger Content

Displays the raw ledger file content.


In [ ]:
# Viewing ledger content

content = bc_client.read_ledger_content()
if content:
    print(content)
